In [1]:
import pandas as pd
import numpy as np

In [2]:
bodiesDF = pd.read_pickle('synchronization/synchronize')
facesDF = pd.read_pickle('faces')
uniqueNames = np.load('uniqueName.npy')

In [3]:
scores = [[] for i in range(len(uniqueNames))]

In [58]:
#Need to adapt the coordinate to the high quality picture
bodiesDF.trackX = bodiesDF.trackX.astype(int) * 2 * 1.7
bodiesDF.trackY = bodiesDF.trackY.astype(int) * 2
bodiesDF.trackH = bodiesDF.trackH.astype(int) * 2 * 2
bodiesDF.trackW = bodiesDF.trackW.astype(int) * 2 * 2

In [59]:
facesDF

,Laurene,Anthony,Wissem,Cynthia,Sabrine,faceX,faceY,rgbSeq
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.211830,1.292989,1.168803,1.270894,1.328637,1039.0,57.0,47.0
3,1.158829,1.242249,1.080242,1.210182,1.315757,1043.0,60.0,48.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.261341,1.332883,1.259527,1.354414,1.299897,1828.0,508.0,58.0


In [64]:
for seq in bodiesDF.rgbSeq.unique():
    bodies = bodiesDF[bodiesDF.rgbSeq == seq]
    faces = facesDF[facesDF.rgbSeq == seq]
    
    #print 'bodies'
    #print bodies
    #print '####'
    #print 'faces'
    #print faces
    for i, f in faces.iterrows():
        if len(bodies) > 0 :        
            
            candidates = bodies[
                [((((bodies.trackX.astype(int) + bodies.trackW.astype(int)) / 2 ) - f.faceX)**2 + 
                (((bodies.trackY.astype(int) + bodies.trackH.astype(int)) / 2 ) - f.faceY)**2) < 
                 (bodies.trackH.astype(int)/2)**2] # 250 arbitrary picked 
                and 
                ((bodies.trackY.astype(int) + bodies.trackH.astype(int)) / 2 ) < f.faceY
            ]
            if not candidates.empty:
                print '******'
                print f
                print candidates
                print '******'

******
Laurene       1.261341
Anthony       1.332883
Wissem        1.259527
Cynthia       1.354414
Sabrine       1.299897
faceX      1828.000000
faceY       508.000000
rgbSeq       58.000000
Name: 9, dtype: float64
        traSeq trackId  trackX  trackY  trackW  trackH         height  \
seq  is                                                                 
1165 4    1165       4     912     150     362     726  1.66906745837   
1166 4    1166       4     930     164     348     696  1.67984271889   

              distance  rgbSeq  
seq  is                         
1165 4   2.32130408287      58  
1166 4   2.46972560883      58  
******
******
Laurene       1.262357
Anthony       1.361335
Wissem        1.265799
Cynthia       1.305505
Sabrine       1.355565
faceX      1086.000000
faceY       422.000000
rgbSeq      262.000000
Name: 139, dtype: float64
        traSeq trackId  trackX  trackY  trackW  trackH         height  \
seq  is                                                        

In [6]:
bodiesDF

,,traSeq,trackId,trackX,trackY,trackW,trackH,height,distance,rgbSeq
seq,is,,,,,,,,,
1155,4,1155,4,233,0,288,576,1.5005300045,1.42739236355,42
1156,4,1156,4,272,13,270,541,1.50847764015,1.56097233295,45
1157,4,1157,4,292,19,262,524,1.5179650116,1.58523178101,47
1158,4,1158,4,334,33,243,487,1.53235393715,1.71000134945,48
1159,4,1159,4,368,44,228,457,1.55183779554,1.81243562698,51
1160,4,1160,4,415,62,206,413,1.57554826161,1.9273519516,52
1161,4,1161,4,423,62,203,406,1.59748577792,2.0393819809,52
1162,4,1162,4,434,67,196,392,1.61563918339,2.07068181038,53
1163,4,1163,4,442,68,192,384,1.63903561843,2.18748092651,54


In [7]:
bodies

,,traSeq,trackId,trackX,trackY,trackW,trackH,height,distance,rgbSeq
seq,is,,,,,,,,,
1157,4,1157,4,292,19,262,524,1.5179650116,1.58523178101,47
